<a href="https://colab.research.google.com/github/michael-korsh/ccprisondata/blob/main/PrisonDataScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cook County Jail Population and COVID-19 Case Scraper**





**Installing and Importing Relevant Libraries**

In [1]:
!java -version
!pip install -q tabula-py
!pip install pygsheets

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
     |████████████████████████████████| 11.7MB 20.7MB/s 
     |████████████████████████████████| 153kB 32.3MB/s 


In [2]:
import tabula
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import requests
import re

import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

**Cook County Jail Population — Retrieval**


In [3]:
def get_jail_data(date):
  jail_data = {}
  
  date_items = date.split('/')
  
  month = date_items[0]
  day = date_items[1]
  year = date_items[2]

  pdf_path = "https://www.cookcountysheriff.org/wp-content/uploads/" + year + '/' + month + '/CCSO_BIU_CommunicationsCCDOC_v1_' + year + '_' + month + '_' + day + '.pdf'
  jail_data["PDF"] = pdf_path

  try:
   df = tabula.read_pdf(pdf_path, pages=1)
  except:
    return None

  jail_data_date = month + '/' + day + '/' + year
  jail_data['Date'] = jail_data_date

  try:
    if df[0]["Under the Custody of the Sheriff"][1] == 'Jail Population':
      jail_pop = int(df[0]['Unnamed: 0'][1].replace(',',''))
      elec_monitoring_pop = int(df[0]['Unnamed: 0'][2].replace(',',''))
    else:
      jail_pop = int(df[0]["Under the Custody of the Sheriff"][1].replace(',',''))
      elec_monitoring_pop = int(df[0]["Under the Custody of the Sheriff"][2].replace(',',''))
  except:
    try:
      df[1]['Under the Custody of the Sheriff'][1] == 'Jail Population'
      jail_pop = int(df[1]['Unnamed: 0'][1].replace(',',''))
      elec_monitoring_pop = int(df[1]['Unnamed: 0'][2].replace(',',''))
    except:
      pass
  
  jail_data["Jail Population"] = jail_pop
  jail_data["Electronic Monitoring"] = elec_monitoring_pop

  return jail_data

**Cook County Jail COVID-19 Cases — Retrieval**




In [4]:
def get_today_covid_data():
  covid_data = {}

  webpage_response = requests.get('https://www.cookcountysheriff.org/covid/covid-19-cases-at-ccdoc/')
  soup = BeautifulSoup(webpage_response.content)

  all_text = soup.find_all('p')
  text_list = []

  for a in all_text:
    text_list.append(a.getText())

  bullets = soup.find_all("li")

  for line in text_list:
    if "As of 5 p.m. on" in line:
      date_list = []
      all_words_in_line = line.split(' ')

      for word in all_words_in_line:
        date_list.append(word)
      date = date_list[5].replace(':','')

      month = int(date.split('/')[0])
      
      if month < 10:
        date = "0" + date
      
      covid_data["Date"] = date

  for li in bullets:
    li_str = li.getText()

    if 'detainees in custody at Cook County Jail are currently positive for COVID-19.' in li_str:
      sentence = li_str.split(' ')
      current_positive = int(sentence[0])
      covid_data['Current Positives'] = current_positive

    if 'detainees in custody are no longer positive and are being monitored at a recovery facility at the jail' in li_str:
      sentence = li_str.split(' ')
      recovered = sentence[0].replace(',','')
      recovered = int(recovered)
      covid_data['Recovered'] = recovered

    if 'detainees have tested negative for COVID-19' in li_str:
      sentence = li_str.split(' ')
      negative = sentence[0].replace(',','')
      negative = int(negative)
      covid_data['Negative'] = negative

    if 'detainees who tested positive for COVID-19 have died while receiving treatment at local hospitals' in li_str:
      sentence = li_str.split(' ')
      detainees_died = sentence[0].replace(',','')
      detainees_died = int(detainees_died)
      covid_data['Detainee Deaths'] = detainees_died

    if 'detainees who are being treated at local hospitals' in li_str:
      sentence = li_str.split(' ')
      detainees_hospitalized = int(sentence[2])
      covid_data['Detainees Hospitalized'] = detainees_hospitalized

  for line in text_list:
    if "Additionally" in line:
      full_sentence = line.split(' ')
      officers_positive = int(full_sentence[1])
      other_employees_positive = int(full_sentence[10])
      covid_data["Sheriff's Staff Positive"] = officers_positive + other_employees_positive
      covid_data["Officers Positive"] = officers_positive

    if "who previously tested positive who are now recovered and have returned to work" in line:
      full_sentence = line.split(' ')
      recovered_employees = int(full_sentence[2].replace(',',''))
      covid_data["Recovered Sheriff's Employees"] = recovered_employees

    if "We are saddened to report that" in line:
      full_sentence = line.split(' ')
      correctional_officer_deaths = int(full_sentence[6])
      covid_data["Correctional Officer Deaths"] = correctional_officer_deaths

  return covid_data

**Helper Functions**

In [5]:
def get_today_jail_data():
  td = datetime.date.today()
  td = td.strftime("%m/%d/%Y")
  return get_jail_data(td)

def reformat_date(date_string):
  date_items = date_string.split('/')
  month = date_items[0].replace('0','')
  if int(date_items[1]) < 10:
    day = date_items[1].replace('0','')
  else:
    day = str(date_items[1])
  year = date_items[2]
  reformatted_date = month + '/' + day + '/' + year
  return reformatted_date

**Initialize Today's Data**

In [6]:
today_covid_data = get_today_covid_data()
today_jail_data = get_today_jail_data()

Got stderr: Jun 11, 2021 2:43:21 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Jun 11, 2021 2:43:21 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jun 11, 2021 2:43:21 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jun 11, 2021 2:43:21 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 11, 2021 2:43:21 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 11, 2021 2:43:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 11, 2021 2:43:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



**Updating Spreadsheets**



*Update Jail Spreadsheet*

In [7]:
def update_jail_spreadsheet(jail_data):

  sh = gc.open('Cook County Jail Population - March 2020')

  if jail_data == None:
    print('Jail data for this day is unavailable.')

  else:
    rf_date_2 = reformat_date(jail_data['Date'])
    print('Jail population data is available for ' + rf_date_2 + '.')
    
    jail_wks = sh[0]
    jail_df = pd.DataFrame(jail_wks.get_all_records())
    
    if rf_date_2 != jail_df.iloc[-1]['Date']:
      new_row = [jail_data['Date'],
                jail_data['Jail Population'],
                jail_data['Electronic Monitoring']]

      cells = jail_wks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
      last_row = len(cells)
      jail_wks = jail_wks.insert_rows(last_row, number=1, values= new_row,inherit=True)
      
      print('Jail population spreadsheet has been updated with data from '+ rf_date_2 + '.')

    else:
      print('Jail population spreadsheet is up to date.')

*Update COVID Spreadsheet*



In [8]:
def update_covid_spreadsheet(today_covid_data):
  
  rf_date = reformat_date(today_covid_data['Date'])

  if today_covid_data:
    print('COVID-19 data is available for ' + rf_date + '.')

  if not today_covid_data:
    print('COVID data for this day is unavailable.')
  
  sh = gc.open('Cook County Jail Population - March 2020')
  covid_wks = sh[1]
  covid_df = pd.DataFrame(covid_wks.get_all_records())

  if rf_date != covid_df.iloc[-1]['Date']:
    jail_wks = sh[0]
    jail_df = pd.DataFrame(jail_wks.get_all_records())
    try:
      jail_df_row = jail_df.loc[jail_df['Date'] == rf_date]
      jail_pop = jail_df_row.iloc[0]['Jail Population']
    except:
      jail_df_row = jail_df.iloc[[0, -1]]
      jail_pop = jail_df_row.iloc[0]['Jail Population']
    today_date = today_covid_data['Date']
    today_positives = today_covid_data['Current Positives']
    recovered_detainees = today_covid_data['Recovered']
    today_negative = today_covid_data['Negative']
    detainee_deaths = today_covid_data['Detainee Deaths']
    co_deaths = today_covid_data['Correctional Officer Deaths']
    today_staff_positive = today_covid_data["Sheriff's Staff Positive"]
    today_co_positive = today_covid_data['Officers Positive']
    recovered_employees = today_covid_data["Recovered Sheriff's Employees"]
    today_infection_rate = (today_positives/jail_pop)*1000

    if 'Detainees Hospitalized' in today_covid_data:
      today_hospitalized == today_covid_data['Detainees Hospitalized']
    else:
      today_hospitalized = 0

    total_detainees_positive = today_positives + recovered_detainees
    total_staff_positive = today_covid_data["Sheriff's Staff Positive"] + today_covid_data["Recovered Sheriff's Employees"]

    new_covid_row = [today_date,
                    total_detainees_positive,
                    today_positives,
                    recovered_detainees,
                    None,
                    None,
                    None,
                    today_negative,
                    today_hospitalized,
                    detainee_deaths,
                    co_deaths,
                    today_staff_positive,
                    today_co_positive,
                    recovered_employees,
                    total_staff_positive,
                    today_infection_rate]

    cells2 = covid_wks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
    last_row = len(cells2)
    covid_wks = covid_wks.insert_rows(last_row, number=1, values= new_covid_row,inherit=True)
    print('COVID-19 spreadsheet has been updated with data from '+ rf_date + '.')

  else:
    print('COVID-19 spreadsheet is up to date.')

**Daily Operations**

In [9]:
def daily_operations():
  update_jail_spreadsheet(today_jail_data)
  update_covid_spreadsheet(today_covid_data)


**RUNNING THE PROGRAM**

*Mondays*

In [10]:
'''
update_jail_spreadsheet(get_jail_data('06/05/2021'))
update_jail_spreadsheet(get_jail_data('06/06/2021'))
'''

"\nupdate_jail_spreadsheet(get_jail_data('06/05/2021'))\nupdate_jail_spreadsheet(get_jail_data('06/06/2021'))\n"

*All Days*

In [11]:
daily_operations()

Jail population data is available for 6/11/2021.
Jail population spreadsheet has been updated with data from 6/11/2021.
COVID-19 data is available for 6/10/2021.
COVID-19 spreadsheet has been updated with data from 6/10/2021.
